In [1]:
import pandas as pd
import numpy as np

# load MATLAB files
from scipy.io import loadmat

#### Load MATLAB datafiles

In [2]:
data = loadmat("data/ex4data1.mat")
data.keys()

dict_keys(['X', '__globals__', '__version__', 'y', '__header__'])

In [4]:
# Add intercept for X
X = np.c_[np.ones((data['X'].shape[0], 1)), data['X']]
y = data['y']

print("X: ", X.shape, "with intercept")
print("y: ", y.shape)

X:  (5000, 401) with intercept
y:  (5000, 1)


In [5]:
weights = loadmat("data/ex3weights.mat")
weights.keys()

dict_keys(['Theta1', 'Theta2', '__globals__', '__version__', '__header__'])

In [6]:
theta1, theta2 = weights['Theta1'], weights['Theta2']
print("Theta1: ", theta1.shape)
print("Theta2: ", theta2.shape)
params = np.r_[theta1.ravel(), theta2.ravel()]
print("Params: ", params.shape)

Theta1:  (25, 401)
Theta2:  (10, 26)
Params:  (10285,)


#### Neural Network Architecture
Input layer size = 400 (20x20 pixels) <br>
Hidden layer size = 25 <br>
Number of labels = 10

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

#### Sigmoid gradient
#### $$ g'(z) = g(z)(1 - g(z))$$
where $$ g(z) = \frac{1}{1+e^{-z}}$$

In [8]:
def sigmoid_gradient(z):
    return sigmoid(z) * (1 - sigmoid(z))

#### Cost Function 
##### Here K is number of classifications corresponding with the output layer and m is the number of training examples
#### $$ J(\theta) = \frac{1}{m}\sum_{i=1}^{m}\sum_{k=1}^{K}\big[-y^{(i)}_{k}\, log\,(( h_\theta\,(x^{(i)}))_k)-(1-y^{(i)}_k)\,log\,(1-h_\theta(x^{(i)}))_k)\big]$$

#### Regularized Cost Function
#### $$ J(\theta) = \frac{1}{m}\sum_{i=1}^{m}\sum_{k=1}^{K}\bigg[-y^{(i)}_{k}\, log\,(( h_\theta\,(x^{(i)}))_k)-(1-y^{(i)}_k)\,log\,(1-h_\theta(x^{(i)}))_k)\bigg] + \frac{\lambda}{2m}\bigg[\sum_{j=1}^{25}\sum_{k=1}^{400}(\Theta_{j,k}^{(1)})^2+\sum_{j=1}^{10}\sum_{k=1}^{25}(\Theta_{j,k}^{(2)})^2\bigg]$$

#### Partial derivative

#### $$ \frac{\delta J(\theta)}{\delta\theta_{j}} = \frac{1}{m}\sum_{i=1}^{m} ( h_\theta (x^{(i)})-y^{(i)})x^{(i)}_{j} + \frac{\lambda}{m}\theta_{j}$$ 
#### Vectorized
#### $$ \frac{\delta J(\theta)}{\delta\theta_{j}} = \frac{1}{m} X^T(g(X\theta)-y) + \frac{\lambda}{m}\theta_{j}$$
##### $$\text{Note: intercept parameter } \theta_{0} \text{ is not to be regularized}$$

In [11]:
def nn_cost_function(params, input_layer_size, hidden_layer_size, num_labels, features, classes, reg):
    theta1 = params[0:(hidden_layer_size * (input_layer_size + 1))].reshape(hidden_layer_size, input_layer_size + 1)
    theta2 = params[hidden_layer_size * (input_layer_size + 1):].reshape(num_labels, (hidden_layer_size + 1))
    
    m = features.shape[0]
    y_matrix = pd.get_dummies(classes.ravel()).as_matrix()
    
    # Cost computation
    a1 = features # 5000x401
    
    z2 = theta1.dot(a1.T) # 25x401 * 401x5000 = 25x5000
    a2 = np.c_[np.ones((features.shape[0], 1)), sigmoid(z2.T)] # 5000x26
    
    z3 = theta2.dot(a2.T) # 10x26 * 26x5000 = 10x5000
    a3 = sigmoid(z3) # 10x5000
    
    # Vectorized implementation
    J = -1/m * np.sum((np.log(a3.T) * y_matrix + np.log(1-a3).T * (1-y_matrix))) + \
        reg/(2*m) * (np.sum(np.square(theta1[:, 1:])) + np.sum(np.square(theta2[:, 1:])))
        
    # Gradients calculation
    d3 = a3.T - y_matrix # 5000x10
    d2 = theta2[:, 1:].T.dot(d3.T) * sigmoid_gradient(z2) # 25x10 x 10x5000 * 25x5000 = 25x5000
    
    delta1 = d2.dot(a1) # 25x5000 x 5000x401 = 25x401
    delta2 = d3.T.dot(a2) # 10x5000 x 5000x26 = 10x26
    
    theta1_ = np.c_[np.ones((theta1.shape[0], 1)), theta1[:, 1:]]
    theta2_ = np.c_[np.ones((theta2.shape[0], 1)), theta2[:, 1:]]
    
    theta1_grad = delta1/m + theta1_*reg/m
    theta2_grad = delta2/m + theta2_*reg/m
    
    return (J, theta1_grad, theta2_grad)

In [13]:
# Regularization parameter = 0
nn_cost_function(params, 400, 25, 10, X, y, 0)[0]

0.28762916516131892

In [14]:
# Regularization parameter = 1
nn_cost_function(params, 400, 25, 10, X, y, 1)[0]

0.38376985909092365